In [1]:
import torch 
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import csv 
import pandas as pd
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler


In [2]:
df = pd.read_csv("dtb.csv")
print(df)
print(df.shape)

class DtSet(Dataset):
    def __init__(self, dataframe):
        self.x = torch.tensor(dataframe.iloc[:, :-1].values, dtype=torch.float32)
        self.y = torch.tensor(dataframe.iloc[:, -1].values, dtype=torch.float32)

    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, ind):
        return self.x[ind], self.y[ind]
    
instance = DtSet(df)

loader = torch.utils.data.DataLoader(instance, batch_size=3, shuffle=True)

print(instance.x, "\n",  instance.y)

     X   Y   Z  res
0    1   2   3    1
1    2   3   4    2
2    3   2   1    3
3    2   1   2    2
4    5   1   1    5
5    4   1   2    4
6   10   1   0   10
7   11   7   6   11
8   15   9   8   15
9   20   9   1   20
10  13   3   3   13
11   3   4   5    3
12   5  13   3    5
13  17  16  16   17
(14, 4)
tensor([[ 1.,  2.,  3.],
        [ 2.,  3.,  4.],
        [ 3.,  2.,  1.],
        [ 2.,  1.,  2.],
        [ 5.,  1.,  1.],
        [ 4.,  1.,  2.],
        [10.,  1.,  0.],
        [11.,  7.,  6.],
        [15.,  9.,  8.],
        [20.,  9.,  1.],
        [13.,  3.,  3.],
        [ 3.,  4.,  5.],
        [ 5., 13.,  3.],
        [17., 16., 16.]]) 
 tensor([ 1.,  2.,  3.,  2.,  5.,  4., 10., 11., 15., 20., 13.,  3.,  5., 17.])


In [3]:
#   MAUVAISE Normalisation des données (Z - score) (creation of noise)

"""for X, Y in loader:
    mean = X.mean()
    std = X.std()
    X = (X - mean) / std"""





'for X, Y in loader:\n    mean = X.mean()\n    std = X.std()\n    X = (X - mean) / std'

In [4]:
# True normalisation (Z - score)
X = df[["X", "Y", "Z"]].values
Y = df[["res"]].values

scaler_x = StandardScaler()
scaler_y = StandardScaler()
X_tensor = torch.tensor(scaler_x.fit_transform(X), dtype = torch.float32)
Y_tensor = torch.tensor(scaler_y.fit_transform(Y), dtype = torch.float32)


print(X_tensor, "\n", Y_tensor)

tensor([[-1.1402, -0.6661, -0.2348],
        [-0.9757, -0.4541,  0.0181],
        [-0.8111, -0.6661, -0.7406],
        [-0.9757, -0.8780, -0.4877],
        [-0.4820, -0.8780, -0.7406],
        [-0.6465, -0.8780, -0.4877],
        [ 0.3409, -0.8780, -0.9935],
        [ 0.5055,  0.3936,  0.5238],
        [ 1.1637,  0.8174,  1.0296],
        [ 1.9866,  0.8174, -0.7406],
        [ 0.8346, -0.4541, -0.2348],
        [-0.8111, -0.2422,  0.2709],
        [-0.4820,  1.6651, -0.2348],
        [ 1.4929,  2.3009,  3.0526]]) 
 tensor([[-1.1402],
        [-0.9757],
        [-0.8111],
        [-0.9757],
        [-0.4820],
        [-0.6465],
        [ 0.3409],
        [ 0.5055],
        [ 1.1637],
        [ 1.9866],
        [ 0.8346],
        [-0.8111],
        [-0.4820],
        [ 1.4929]])


In [5]:
class LinRegModel(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = nn.Linear(in_features, 3),
        self.linear = nn.Linear(3, 6),
        self.linear = nn.Linear(6, 6),
        self.linear = nn.Linear(6, 9),
        self.linear = nn.Linear(9, 12),
        self.linear = nn.Linear(12, 12),
        self.linear = nn.Linear(12, 9),
        self.linear = nn.Linear(9, 9),
        self.linear = nn.Linear(9, 7),
        self.linear = nn.Linear(7, 5),
        self.linear = nn.Linear(5, 3),
        self.linear = nn.Linear(3, 1)


    def forward(self, x):
        return self.linear(x)

In [6]:
in_features = 3
out_features = 1
model = LinRegModel(in_features, out_features)

In [7]:
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr = 0.1)


In [8]:
nb_epochs = 100

for i in range(nb_epochs):
    outputs = model(X_tensor)
    loss = criterion(outputs, Y_tensor)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    print(f'Epoch [{i+ 1}], Loss: {loss.item():.9f}')



Epoch [1], Loss: 0.480779469
Epoch [2], Loss: 0.291058570
Epoch [3], Loss: 0.202426270
Epoch [4], Loss: 0.152074233
Epoch [5], Loss: 0.118646644
Epoch [6], Loss: 0.094297275
Epoch [7], Loss: 0.075718120
Epoch [8], Loss: 0.061227698
Epoch [9], Loss: 0.049802382
Epoch [10], Loss: 0.040736280
Epoch [11], Loss: 0.033508513
Epoch [12], Loss: 0.027722200
Epoch [13], Loss: 0.023070483
Epoch [14], Loss: 0.019314485
Epoch [15], Loss: 0.016267585
Epoch [16], Loss: 0.013783648
Epoch [17], Loss: 0.011748016
Epoch [18], Loss: 0.010070586
Epoch [19], Loss: 0.008680413
Epoch [20], Loss: 0.007521521
Epoch [21], Loss: 0.006549647
Epoch [22], Loss: 0.005729709
Epoch [23], Loss: 0.005033827
Epoch [24], Loss: 0.004439768
Epoch [25], Loss: 0.003929747
Epoch [26], Loss: 0.003489494
Epoch [27], Loss: 0.003107487
Epoch [28], Loss: 0.002774417
Epoch [29], Loss: 0.002482694
Epoch [30], Loss: 0.002226122
Epoch [31], Loss: 0.001999605
Epoch [32], Loss: 0.001798930
Epoch [33], Loss: 0.001620589
Epoch [34], Loss: 0

In [ ]:

new_x = np.array([[5223, 1, 2025]])
scalar_test = StandardScaler()
new_x_tensor = torch.tensor((new_x), dtype = torch.float32)

model.eval()
with torch.no_grad():
    prediction_norm = model(new_x_tensor)

prediction_denorm = prediction_norm.item() 

print(f"Predicted for X = {new_x}: {(prediction_denorm)}")f

Predicted for X = [[5223    1 2025]]: 5214.49560546875
